In [1]:
# %load ../jupyter_setup.txt
# Convenient jupyter setup
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False 

# 1. Imports

In [2]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import rioxarray as rxr
import geopandas as gpd

import pylandstats as pls

from src.models.geograph import GeoGraph
from src.data_loading.rasterio_utils import polygonise
from src.constants import GWS_DATA_DIR, ESA_LANDCOVER_DIR, UTM35N

# Parse geotif landcover data
chernobyl_path = lambda year: GWS_DATA_DIR / "esa_cci_rois" / f"esa_cci_{year}_chernobyl.tif" 

# Parse ROIS
rois = gpd.read_file(GWS_DATA_DIR / "chernobyl_exclusion_zone_v1.geojson")
cez = rois[rois["name"] == "Chernobyl Exclusion Zone"]

/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# 2. Loading Chernobyl Landcover data (ESA CCI)

For this demonstration we will use the ESA CCI landcover dataset as an example to illustrate how
geographs enable us to reproduce the same metrics that traditional fragmentation software (pylandstats, fragstats)
produce.

In this demo we will look specifically at the Chernobyl exclusion zone, so we reproject the satellite data to the right coordinate
system (UTM35N) and clip to the CEZ region of interest).

In [3]:
def clip_and_reproject(xrdata, clip_geometry=None, to_crs=UTM35N, x_res=300, y_res=300):
    
    if clip_geometry is not None:
        clipped_data = xrdata.rio.clip(clip_geometry)
    else:
        clipped_data = xrdata
        
    if to_crs is not None:
        reprojected_data = clipped_data.rio.reproject(to_crs, resolution=(x_res, y_res))
    else:
        reprojected_data = clipped_data
    
    return reprojected_data

In [ ]:
# Loading raster data
years = range(2000, 2015)

rasters = {year: clip_and_reproject(rxr.open_rasterio(chernobyl_path(year)), 
                                    clip_geometry=cez.geometry) 
           for year in years}

In [ ]:
%%time
# Loading raster data into geographs
graphs = {year: GeoGraph(data=rasters[year].data, 
                         transform=rasters[year].rio.transform(), 
                         crs=UTM35N, 
                         connectivity=8)
          for year in years}

In [ ]:
# Loading raster data into pylandstats
landscapes = {year: pls.Landscape(rasters[year].data.squeeze(), 
                                  res=(300,300) , 
                                  nodata=0) 
              for year in years}

# 3. Metrics calculation

In [ ]:
from src.models.geotimeline import GeoGraphTimeline

In [ ]:
graph_timestack = GeoGraphTimeline(graphs)

In [ ]:
graph_timestack.get_metric(total_area)